In [2]:
import pandas as pd

In [49]:
joined = pd.read_csv('joined')

In [58]:
first_query = pd.read_csv('/Users/drewrice/Desktop/first_query.txt', header=None)
first_query.rename(columns = {0:'First Name',1:'Last Name',2:'YearsOfExp', 3: 'State',4:'Zip',5:'Rating'},inplace=True)
first_query.head()

,First Name,Last Name,YearsOfExp,State,Zip,Rating
0,EZDoctor,Virtual Visit,0,FL,00000,NaN
1,Joseph,Abapo,27,CA,92505,NaN
2,Mark,Knight,14,CA,92869,4.0
3,YA,YA,0,TX,77027,5.0
4,Enenge,Abodjedi,21,CT,06902,4.0


In [106]:
merged = pd.merge(joined,first_query,on=['First Name','Last Name', 'State'], how='left')

In [107]:
merged['YearsOfExp'].fillna(0,inplace=True)
merged['Rating'].fillna(0,inplace=True)

In [124]:
# drop rows with any NaN values
# not a long-term solution, but a quick and dirty model
no_suit = merged[merged.indem_label == 0]
suit_no_indem = merged[merged.indem_label == 1]
suit_indem = merged[merged.indem_label == 2]

no_suit.dropna(how='any', inplace=True)
suit_no_indem.dropna(how='any', inplace=True)
suit_indem.dropna(how='any', inplace=True)

# print totals after NaN drop
print no_suit.shape[0], suit_indem.shape[0], suit_no_indem.shape[0]

# !
# change values to 1 for binary classification
# suit_indem['indem_label'] = 1

# random sample from the three categories
no_suit = no_suit.sample(500)
suit_no_indem = suit_no_indem.sample(164)
suit_indem = suit_indem.sample(24)

1205 24 164


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [125]:
# combine random sample dataframes into concat_df
concat_df = pd.concat([no_suit, suit_indem, suit_no_indem], axis=0)

In [126]:
# build label and feature dataframes
y = concat_df.pop('indem_label')
to_model = concat_df[['BoardCredit','ClaimsBand','Procedures','JCAHO','age','RecordKepping']]
state = concat_df.pop('State')
dummies = pd.get_dummies(state)
to_model = pd.concat([to_model,dummies],axis=1)

In [127]:
# train / test split with random state
X_train, X_test, y_train, y_test = train_test_split(to_model, y, test_size=0.10)

In [128]:
model = RandomForestClassifier(n_estimators=64,oob_score=True)

In [129]:
# fit, validate with OOB score
model.fit(X_train,y_train)
model.oob_score_

0.72536348949919227

In [130]:
preds = model.predict(X_test)
model.score(X_test, y_test)

0.79710144927536231

In [131]:
confusion_matrix(y_test, preds)

array([[47,  6,  2],
       [ 5,  7,  0],
       [ 1,  0,  1]])